In [ ]:
# STEP 1: Install required packages
!pip install --quiet beautifulsoup4 google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

# STEP 2: Upload .eml files and Google credentials
from google.colab import files

print("📤 Upload .eml files AND your Google Service Account JSON file (credentials)")
uploaded = files.upload()  # Upload .eml and credentials.json


📤 Upload .eml files AND your Google Service Account JSON file (credentials)


Saving tranquil-leaf-463918-a5-3a521c78f0ec.json to tranquil-leaf-463918-a5-3a521c78f0ec.json
Saving ⭐ NEW JULY FOB  ⚡ Vietnam to INDIA ⚡ .eml to ⭐ NEW JULY FOB  ⚡ Vietnam to INDIA ⚡  (2).eml
Saving CIMC LOGISTICS VIETNAM __ OFFER SEA FREIGHT __ VIETNAM - INDIA __ JUNE, 2025 (1).eml to CIMC LOGISTICS VIETNAM __ OFFER SEA FREIGHT __ VIETNAM - INDIA __ JUNE, 2025 (1) (2).eml
Saving Re_ Greeting from JHSHIP--China.eml to Re_ Greeting from JHSHIP--China (2).eml
Saving Special Rates to Mondra —Let’s Connect at JC Trans Dubai (1).eml to Special Rates to Mondra —Let’s Connect at JC Trans Dubai (1) (2).eml
Saving UPDATE RATE FROM HAIPHONG - INDIA, 19_JUN, MAYWELL VIETNAM (2).eml to UPDATE RATE FROM HAIPHONG - INDIA, 19_JUN, MAYWELL VIETNAM (2) (2).eml
Saving Competitive prices _ From China to your city port _ Week 25 rate (1).eml to Competitive prices _ From China to your city port _ Week 25 rate (1) (2).eml
Saving DIRECT ROUTING __ LIMTED OFFER __ HCM - NSA __ RCL (1).eml to DIRECT ROUTING __

In [ ]:
# STEP 3: Parse and extract from emails
import email
from email import policy
from email.parser import BytesParser
from bs4 import BeautifulSoup
import pandas as pd

def parse_eml(file_data):
    msg = BytesParser(policy=policy.default).parsebytes(file_data)
    sender = msg['From']
    date = msg['Date']
    subject = msg['Subject']

    # Get body
    content = ""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':
                content += part.get_content()
    else:
        content = msg.get_content()

    return sender, date, subject, content

def extract_quotations(content):
    soup = BeautifulSoup(content, 'html.parser')
    text = soup.get_text(separator='\n')
    lines = []

    for line in text.split('\n'):
        line = line.strip()
        parts = line.split()
        if len(parts) >= 6 and any(port in line.upper() for port in ['SHANGHAI', 'HAIPHONG', 'HCM', 'NINGBO', 'TIANJIN']):
            lines.append({
                'Origin Port': parts[0],
                'Shipping Line': parts[1],
                '20GP Rate': parts[2],
                '40HC Rate': parts[3],
                'ETD / Cutoff': parts[4],
                'T/S': parts[5]
            })
    return lines


In [ ]:
# STEP 4: Parse all uploaded files
records = []

for fname in uploaded:
    if fname.endswith(".eml"):
        with open(fname, 'rb') as f:
            data = f.read()
            sender, date, subject, content = parse_eml(data)
            quotations = extract_quotations(content)
            for q in quotations:
                q.update({'Sender': sender, 'Date': date})
                records.append(q)

df = pd.DataFrame(records)
df.head()


,Origin Port,Shipping Line,20GP Rate,40HC Rate,ETD / Cutoff,T/S,Sender,Date
0,*,HAIPHONG,to,NHAVA,SHEVA,/,GLENN/CIMC <glenn@cimclogistics.vn>,"Thu, 19 Jun 2025 09:45:24 +0530"
1,SHANGHAI,CARRIER,20GP,40HC,ETD,T/SNSA,Mason <sal19@nbjhship.com.cn>,"Thu, 19 Jun 2025 08:54:24 +0530"
2,ZIM,2500,2500,23.JUNE,NONINGBO,CARRIER,Mason <sal19@nbjhship.com.cn>,"Thu, 19 Jun 2025 08:54:24 +0530"
3,2200,2200,25/27/28.JUNE,NOTIANJIN,CARRIER,20GP,Mason <sal19@nbjhship.com.cn>,"Thu, 19 Jun 2025 08:54:24 +0530"
4,ADD:4TH,"FLOOR,NO.77",HEYI,RD.HAISHU,DISTRICT.NINGBO,CHINA,Mason <sal19@nbjhship.com.cn>,"Thu, 19 Jun 2025 08:54:24 +0530"


In [ ]:
# STEP 5: Connect to Google Sheets & Upload

from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# 👉 Replace with your actual sheet ID
SHEET_ID = "121twJ-cP73feuV7ZDJNeFcQWWdBv3BOTcTspJtvbeJY"
SHEET_NAME = "Sheet1"
CREDENTIALS_FILE = [f for f in uploaded if f.endswith('.json')][0]
print("✅ Using credentials file:", CREDENTIALS_FILE)


def upload_to_sheet(df, sheet_id, sheet_name, credentials_file):
    creds = Credentials.from_service_account_file(
        credentials_file,
        scopes=["https://www.googleapis.com/auth/spreadsheets"]
    )
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()

    values = [df.columns.tolist()] + df.values.tolist()

    body = {"values": values}
    sheet.values().update(
        spreadsheetId=sheet_id,
        range=f"{sheet_name}!A1",
        valueInputOption="RAW",
        body=body
    ).execute()
    print("✅ Data uploaded to Google Sheet successfully.")

upload_to_sheet(df, SHEET_ID, SHEET_NAME, CREDENTIALS_FILE)


✅ Using credentials file: tranquil-leaf-463918-a5-3a521c78f0ec.json
✅ Data uploaded to Google Sheet successfully.
